# Error metrics 

In [2]:
from configparser import ConfigParser
from sqlalchemy import create_engine
import pandas as pd
import pandas_profiling
from ua_parser import user_agent_parser
from geolite2 import geolite2
import geopandas as gpd
import pycountry
import re
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(color_codes=True)
sns.set_palette('autumn')
from pandas.api.types import is_numeric_dtype


def read_db_config(filename='config-dump.ini', section='mysql'):
    """ Read database configuration file and return a dictionary object
    :param filename: name of the configuration file
    :param section: section of database configuration
    :return: a dictionary of database parameters
    """
    # create parser and read ini configuration file
    parser = ConfigParser()
    parser.read(filename)
 
    # get section, default to mysql
    db = {}
    if parser.has_section(section):
        items = parser.items(section)
        for item in items:
            db[item[0]] = item[1]
    else:
        raise Exception('{0} not found in the {1} file'.format(section, filename))
 
    return db

dbconfig = read_db_config()
CONN = 'mysql+mysqlconnector://'+dbconfig['user']+':'+dbconfig['pwd']+'@'+dbconfig['host']+':'+dbconfig['port'] + '/'+dbconfig['dbname']
engine = create_engine(CONN)

# Supress scientific notation 
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Select keystroke data from a a sample of 10 pps 

In [128]:
ts_sample_q = 'select * from TEST_SECTIONS where TEST_SECTION_ID in (3311,7066,8588,25430,37943,40743,42758,44499,44881,46533,47133,50500,52197,53747,55365,58228,58900,59005,63115,63797,66154,68815,70095,70457,75722,79114,81804,82144,83149,84707,87964,88311,89209,90929,92002,92665,93667,94581,95383,100762,103385,109026,111945,112517,119408,126143,126606,131524,133819,136756,143355,145228,150465,152594,154920,156058,156873,156838,158021,158898,160406,162405,161025,166558,168478,170456,170549,172584,178121,179449,182345,183047,183007,183097,183649,184508,186512,190178,192069,194430,194983,197419,198564,204421,206255,207502,209868,210538,210811,210827,211526,211569,212962,213266,214818,215439,219773,225393,228384,245909,243218,244440,249313,251070,256776,257294,258751,259926,265397,269776,272100,275778,277831,283891,284886,289331,292060,296108,299750,302044,304951,306042,306269,308393,313931,313904,314293,317137,320599,321014,321708,322190,323258,324494,325925,327011,329328,331115,333568,336576,340421,343962,344034,344258,345612,347040,348963,354387,354739,354893,355255,358817,360363,364085,365394,366450,366425,370877,371878,372184,372234,375474,377128,377758,381549,381369,381174,382051,383313,384877)'
ts_sample = pd.read_sql_query(ts_sample_q, engine)
ts_sample.fillna(0, inplace=True)

In [129]:
ts_sample.head()

TEST_SECTION_ID  SENTENCE_ID  PARTICIPANT_ID  \
0             3311         1253             842   
1             7066         1470            2168   
2             8588          889            2610   
3            25430         1341            7733   
4            37943         1263           11443   

                                      USER_INPUT  INPUT_TIME  EDIT_DISTANCE  \
0                 The night promised to be long.       14217              0   
1              Indy time trials begin Saturday .        9958              1   
2              I just had my head in the clouds.       13245              0   
3  It’s all about naking the whole story public.       18337              2   
4          hey would play child in Bruno's Aires        5672              7   

   ERROR_RATE    WPM  INPUT_LENGTH  ERROR_LEN  POTENTIAL_WPM  \
0       0.000 24.478            29         30         32.345   
1       3.030 38.562            32         33         47.268   
2       0.000 28.992            32         33         36.533   
3       4.444 28.794            44         45         22.948   
4      18.421 76.164            36         38         81.839   

   POTENTIAL_LENGTH  DEVICE  INPUT_STREAM  
0                15  mobile             0  
1                31  mobile             0  
2                38  mobile             0  
3                12  mobile             0  
4                35  mobile             0

In [130]:
participants = pd.unique(ts_sample.PARTICIPANT_ID)
sentences = pd.unique(ts_sample.SENTENCE_ID)

In [131]:
ks_sample_q = 'select * from KEYSTROKES where KEYCODE != 229 and TEST_SECTION_ID in (3311,7066,8588,25430,37943,40743,42758,44499,44881,46533,47133,50500,52197,53747,55365,58228,58900,59005,63115,63797,66154,68815,70095,70457,75722,79114,81804,82144,83149,84707,87964,88311,89209,90929,92002,92665,93667,94581,95383,100762,103385,109026,111945,112517,119408,126143,126606,131524,133819,136756,143355,145228,150465,152594,154920,156058,156873,156838,158021,158898,160406,162405,161025,166558,168478,170456,170549,172584,178121,179449,182345,183047,183007,183097,183649,184508,186512,190178,192069,194430,194983,197419,198564,204421,206255,207502,209868,210538,210811,210827,211526,211569,212962,213266,214818,215439,219773,225393,228384,245909,243218,244440,249313,251070,256776,257294,258751,259926,265397,269776,272100,275778,277831,283891,284886,289331,292060,296108,299750,302044,304951,306042,306269,308393,313931,313904,314293,317137,320599,321014,321708,322190,323258,324494,325925,327011,329328,331115,333568,336576,340421,343962,344034,344258,345612,347040,348963,354387,354739,354893,355255,358817,360363,364085,365394,366450,366425,370877,371878,372184,372234,375474,377128,377758,381549,381369,381174,382051,383313,384877)'
ks_sample = pd.read_sql_query(ks_sample_q, engine)
ks_sample.fillna(0, inplace=True)

In [132]:
ks_sample.head()

KEYSTROKE_ID     PRESS_TIME   RELEASE_TIME LETTER  TEST_SECTION_ID  \
0        142728  1537449225028  1537449225078      T             3311   
1        142729  1537449225209  1537449225228      h             3311   
2        142730  1537449225390  1537449225407      e             3311   
3        142731  1537449225708  1537449225738                    3311   
4        142732  1537449226613  1537449226644      n             3311   

   KEYCODE     IKI  KPD  
0       84 444.000   50  
1       72 181.000   19  
2       69 181.000   17  
3       32 318.000   30  
4       78 512.000   31

In [133]:
s_sample_q = 'select * from SENTENCES where SENTENCE_ID in (1253,1470,889,1341,1263,431,736,491,223,1162,602,797,137,170,823,535,1433,1461,1368,768,1309,1343,1359,555,713,75,490,1521,1028,861,182,1460,270,1261,433,80,512,203,475,541,1254,652,1165,1212,1182,1056,373,1142,1097,679,961,506,997,118,51,1020,911,641,561,1275,1158,846,1141,901,472,167,447,767,1146,705,209,657,720,685,184,738,1068,1100,73,192,1015,1016,1315,990,440,857,1384,235,1383,187,1205,849,874,169,1189,802,669,892,704,670,831,625,357,804,528,1298,1317,994,1251,982,1400,867,1202,810,928,1366,1022,208,295,746,1402,23,414,1167,1408,1050,1130,817,835,1093,1223,939,1365,771,204,312,1191,252,620,1417,1064,876,93,848,384,698,912,1059,509,1211,176,1206,304,1104,161,1143,289,999,21,1081)'
s_sample = pd.read_sql_query(s_sample_q, engine)
s_sample.fillna(0, inplace=True)
s_sample.head()

SENTENCE_ID                                           SENTENCE  SOURCE
0           21                           Email the consent to me.  mobile
1           23                  No, at least not to my knowledge.  mobile
2           51                         Just bought out Toys R Us.  mobile
3           73  I would suggest you and Mark address this toge...  mobile
4           75                         Still waiting on decision.  mobile

In [147]:
participants

array([  842,  2168,  2610,  7733, 11443, 12245, 12954, 13444, 13529,
       13896, 14106, 15071, 15541, 15904, 16390, 17348, 17511, 17553,
       18695, 18873, 19368, 19874, 20166, 20245, 21638, 22395, 23095,
       23202, 23409, 23753, 24421, 24485, 24675, 25184, 25426, 25606,
       25875, 26194, 26418, 27814, 28378, 29812, 30419, 30632, 32586,
       34471, 34603, 35999, 36511, 37540, 39245, 39814, 41392, 41828,
       42317, 42572, 42731, 42647, 42944, 43163, 43418, 43679, 43650,
       45159, 45587, 45949, 46002, 46443, 47478, 47860, 48449, 48526,
       48515, 48572, 48703, 48883, 49444, 50518, 50912, 51434, 51584,
       52118, 52462, 54149, 54657, 54959, 55498, 55649, 55710, 55714,
       55866, 55872, 56237, 56315, 56557, 56771, 57767, 59363, 60109,
       64013, 64358, 62957, 65689, 66209, 67876, 68009, 68412, 68829,
       70066, 71123, 71828, 72749, 73040, 73762, 73909, 74500, 74846,
       75372, 75857, 76110, 76504, 76687, 76724, 77071, 77781, 77775,
       77821, 78139,

In [228]:
import xml.etree.cElementTree as ET
from xml.sax.saxutils import escape, unescape

for p in range(0, len(participants)):
    p_id = participants[p]   
    ts_ids = pd.unique(ts_sample[ts_sample.PARTICIPANT_ID==p_id].TEST_SECTION_ID)
    #print(ts_ids)
    
    ks_ids = ks_sample[ks_sample.TEST_SECTION_ID == ts_ids[0]].KEYSTROKE_ID.values
    #print(ks_ids)
    
    if len(ks_ids > 0):
        start_time = ks_sample[ks_sample.KEYSTROKE_ID==ks_ids[0]]['PRESS_TIME'].values[0]*10000    
        
        root = ET.Element("TextTest", trials=str(len(ts_ids)), ticks=str(start_time))

        valid=False
        for i in range(0, len(ts_ids)):
            ts_id = ts_ids[i]
            s_id = ts_sample[ts_sample.TEST_SECTION_ID==ts_id].SENTENCE_ID.values[0]
            sentence = s_sample[s_sample.SENTENCE_ID==s_id].SENTENCE.values[0]
            ks_ids = ks_sample[ks_sample.TEST_SECTION_ID == ts_id].KEYSTROKE_ID.values
            ks = ks_sample[ks_sample.KEYSTROKE_ID==ks_ids[-1]]['LETTER'].values[0].lower()
            if ks!= ' ':
                doc = ET.SubElement(root, "Trial", testing='true', number=str(i+1), entries=str(len(ks_ids)))
            else:
                doc = ET.SubElement(root, "Trial", testing='true', number=str(i+1), entries=str(len(ks_ids)-1))
            
            if len(sentence.strip())<= len(ks_ids):
                valid=True
                ET.SubElement(doc, "Presented").text=sentence
                for j in range(0, len(ks_ids)):
                    ks_id = ks_ids[j]
                    ks = ks_sample[ks_sample.KEYSTROKE_ID==ks_id]['LETTER'].values[0].lower()
                    if j==len(ks_ids)-1 and ks==' ':
                        ''
                    else:
                        if ks=='bksp':
                            ks = '&#8;'
                           
                        if ks=='&#8;' or len(ks)==1:
                            time = ks_sample[ks_sample.KEYSTROKE_ID==ks_id]['PRESS_TIME'].values[0]*10000
                            ET.SubElement(doc, "Entry", char=str(ks), ticks=str(time))
                        
                ET.SubElement(doc, "Transcribed").text=ts_sample[ts_sample.TEST_SECTION_ID==ts_id].USER_INPUT.values[0]

        if valid:
            tree = ET.ElementTree(root)
            tree.write("participant_"+str(p_id)+".xml", encoding="utf-8")

In [224]:
ks_ids

array([], dtype=int64)

In [216]:
unescape('&amp;#8;')

'&#8;'

In [174]:
for child in root:
    print(child.tag, child.attrib)

Trial {'testing': 'true', 'number': '1', 'entries': '24'}


#### One way to investigate types of errors committed by participants requires reconstruction of input streams of each test section from keystroke-level data. Only data from Mac devives (IPhone, IPad) can be used for this purpose, since for these devives keycodes on soft keyboard are informative. 

In [ ]:
# Check: There is no Mac device where KEYCODE is undefined (229)
"""
select * from KEYSTROKES 
where test_section_id in (
	select TEST_SECTION_ID 
	from FIRST_15_TEST_SECTIONS 
	where PARTICIPANT_ID in (
		select PARTICIPANT_ID 
		from PARTICIPANTS
		where BROWSER LIKE '%Mac%'
	) 
) and KEYCODE = 229 
"""

# p.s. There are SOME non-Mac devices where KEYCODE is NOT undefined: 
"""
select distinct BROWSER 
from PARTICIPANTS where 
PARTICIPANT_ID in (
	select PARTICIPANT_ID from FIRST_15_TEST_SECTIONS
	where TEST_SECTION_ID in (
		select distinct TEST_SECTION_ID from KEYSTROKES 
		where test_section_id in (
			select TEST_SECTION_ID 
			from FIRST_15_TEST_SECTIONS 
			where PARTICIPANT_ID in (
				select PARTICIPANT_ID 
				from PARTICIPANTS
				where BROWSER NOT LIKE '%Mac%'
			) 
		) and KEYCODE != 229
	)
)
"""

# (1587 different browser strings, see data/nonmac_browsers_with_defined_keycode.csv)

## Input stream reconstruction

In [ ]:
# Make two versions of the INPUT_STREAM 

# This code concatenates all KEYSTROKES in the sequence, filtering out special characters 
"""
ALTER TABLE FIRST_15_TEST_SECTIONS ADD COLUMN INPUT_STREAM TEXT;

UPDATE FIRST_15_TEST_SECTIONS ts
set INPUT_STREAM = (
    SELECT STREAM FROM (
        SELECT 
        	TEST_SECTION_ID, 
        	GROUP_CONCAT(LETTER ORDER BY PRESS_TIME SEPARATOR '') as STREAM
        	
        FROM KEYSTROKES
        WHERE 
        	KEYCODE not in (229, 8, 9, 16, 17, 18, 19, 20, 27, 33, 34, 35, 36, 37, 38, 39, 40, 42, 44, 45, 46, 91, 93, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 144, 145)
        	and 
        	TEST_SECTION_ID in (
        		select TEST_SECTION_ID from FIRST_15_TEST_SECTIONS
        	)
        GROUP BY TEST_SECTION_ID
    ) as gr
    where ts.TEST_SECTION_ID = gr.TEST_SECTION_ID 
)
"""

# This code does NO filtering. Special characters will be (later) replaced with symbols

"""
ALTER TABLE FIRST_15_TEST_SECTIONS ADD COLUMN INPUT_STREAM_FULL TEXT;

UPDATE FIRST_15_TEST_SECTIONS ts
set INPUT_STREAM_FULL = (
    SELECT STREAM FROM (
        SELECT 
        	TEST_SECTION_ID, 
        	GROUP_CONCAT(LETTER ORDER BY PRESS_TIME SEPARATOR '') as STREAM
        	
        FROM KEYSTROKES
        WHERE 
        	#KEYCODE not in (229, 8, 9, 16, 17, 18, 19, 20, 27, 33, 34, 35, 36, 37, 38, 39, 40, 42, 44, 45, 46, 91, 93, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 144, 145)
        	#and 
        	TEST_SECTION_ID in (
        		select TEST_SECTION_ID from FIRST_15_TEST_SECTIONS
        	)
        GROUP BY TEST_SECTION_ID
    ) as gr
    where ts.TEST_SECTION_ID = gr.TEST_SECTION_ID 
)
"""

# Excluded special characters:  see keycodes.js in the typing test source files 
# (229, 8, 9, 16, 17, 18, 19, 20, 27, 33, 34, 35, 36, 37, 38, 39, 40, 42, 44, 45, 46, 91, 93, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 144, 145)

In [ ]:
# Replace BKSP with < and SHIFT with ^
"""
UPDATE FIRST_15_TEST_SECTIONS ts
set INPUT_STREAM_FULL = ( REPLACE(REPLACE(INPUT_STREAM_FULL, 'SHIFT', '^'), 'BKSP', '<') )
"""

# Replace all other special keys with ± 
"""
UPDATE FIRST_15_TEST_SECTIONS ts
#set INPUT_STREAM_FULL = REGEXP_REPLACE(INPUT_STREAM_FULL,'(ARw_DOWN)|(ARW_LEFT)|(ARW_RIGHT)|(ARW_UP)|(CAPS_LOCK)|(CTRL)|(DELETE)|(ESC)|(F1)|(F10)|(F11)|(F12)|(F8)|(HOME)|(INSERT)|(NUM_LK)|(PAUSE)|(TAB)','±'); 
set INPUT_STREAM_FULL = REPLACE(
 REPLACE(
 REPLACE(
 REPLACE(
 REPLACE(
 REPLACE(
 REPLACE(
 REPLACE(
 REPLACE(
 REPLACE(
 REPLACE(
 REPLACE(
 REPLACE(
 REPLACE(
 REPLACE(
 REPLACE(
 REPLACE(
 REPLACE(
 REPLACE(INPUT_STREAM_FULL,'ALT','±'),'TAB','±'),'PAUSE','±'),'NUM_LK','±'),'INSERT','±'),'HOME','±'),'F8','±'),'F12','±'),'F11','±'),'F10','±'),'F1','±'),'ESC','±'),'DELETE','±'),'CTRL','±'),'CAPS_LOCK','±'),'ARW_UP','±'),'ARW_RIGHT','±'),'ARW_LEFT','±'),'ARw_DOWN','±'); 

"""

# REGEX (only in MySQL 8.0.12 :( ): (ARw_DOWN)|(ARW_LEFT)|(ARW_RIGHT)|(ARW_UP)|(CAPS_LOCK)|(CTRL)|(DELETE)|(ESC)|(F1)|(F10)|(F11)|(F12)|(F8)|(HOME)|(INSERT)|(NUM_LK)|(PAUSE)|(TAB)


### Now special characters are replaced: 
#### BKSP with <
#### SHIFT with ^
#### (OTHER) with ±

In [ ]:
# How to count Correct (C), Fixes (F), Incorrect Fixed (IF) ? 

# Correct (C) is the number of correct characters in the transcribed text.  <- IS_CORRECT

# Fixes (F) are keystrokes in the input stream, which are
# edit functions (backspace, delete, cursor movement, etc), 
# modifier keys (shift, alt, control, ets), or 
# navigation leys (left, cight, mouse click, etc). <- IS_FIXES

# Incorrect Fixed (IF) keystrokes are those in the input stream that are 
# not editing keys (F), but which 
# do not appear in the final transcribed text result. <- IS_INCFIXED


# Fixes (IS_FIXES) 
"""
ALTER TABLE FIRST_15_TEST_SECTIONS ADD COLUMN IS_FIXES INT;

UPDATE FIRST_15_TEST_SECTIONS ts
set IS_FIXES = (
    SELECT CNT FROM (
        SELECT 
            TEST_SECTION_ID, 
            COUNT(KEYSTROKE_ID) as CNT

        FROM KEYSTROKES
        WHERE 
            KEYCODE in (8, 9, 16, 17, 18, 19, 20, 27, 33, 34, 35, 36, 37, 38, 39, 40, 42, 44, 45, 46, 91, 93, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 144, 145)
            and 
            TEST_SECTION_ID in (
                select TEST_SECTION_ID from FIRST_15_TEST_SECTIONS
            )
        GROUP BY TEST_SECTION_ID
    ) as gr
    where ts.TEST_SECTION_ID = gr.TEST_SECTION_ID 
)
"""

# Incorrect Fixed (IS_INCFIXED)

# Correct (IS_CORRECT)

select INPUT_STREAM from FIRST_15_TEST_SECTIONS
MINUS 
select USER_INPUT from FIRST_15_TEST_SECTIONS


#### Can't find any MySQL 5.7 function to compare strings as sets (character-wise). Python then.

In [ ]:
# Query Transctibed Text (T): USER_INPUT and 
# user Input Stream (without special characters, IS): INPUT_STREAM, 
# Convert to sets and subtract these sets, 
# Length of the difference is Incorrect Fixed (IF) : IS_INCFIXED

# 

In [92]:
Q = """
select 
    TEST_SECTION_ID, 
    USER_INPUT, 
    INPUT_STREAM,
    INPUT_STREAM_FULL
from 
    FIRST_15_TEST_SECTIONS
"""

# Load data
input_strings = pd.read_sql_query(Q, engine)
input_strings.fillna('',inplace=True)
input_strings.tail()

TEST_SECTION_ID                                         USER_INPUT  \
126827           386163  US vessels will form two separate training units.   
126828           386166                             She has a web address.   
126829           386167  These factors dictate clear limits to the conc...   
126830           386170  Its revenues last year reached 3.2 billion dol...   
126831           386173                 They have not issued an order yet.   

                                             INPUT_STREAM  \
126827  US vessels will form two separate training units.   
126828                             She has a web address.   
126829  These factors dictate clear limits to the conc...   
126830  TIta sts revenues last year reached 3.2 billio...   
126831               They hsave not issued oan order yet.   

                                        INPUT_STREAM_FULL  
126827  ^^U^S vessels will form two separate training ...  
126828                            ^She has a web address.  
126829  ^These factors dictate clear limits to the con...  
126830  ^T<^Ita <<<s<ts revenues last year reached 3.2...  
126831            ^They hs<ave not issued o<an order yet.

In [95]:
input_strings['INPUT_STREAM_FULL_LIST'] = input_strings.INPUT_STREAM_FULL.apply(lambda x: set(x))
input_strings['INPUT_STREAM_LIST'] = input_strings.INPUT_STREAM.apply(lambda x: set(x))
input_strings['USER_INPUT_LIST'] = input_strings.USER_INPUT.apply(lambda x: set(x))
input_strings['INCORRECT_FIXED'] = input_strings.INPUT_STREAM_LIST.subtract(input_strings.USER_INPUT_LIST)
input_strings['FIXES'] = input_strings.INPUT_STREAM_FULL_LIST.subtract(input_strings.INPUT_STREAM_LIST)
input_strings['CORR'] = [(set(a).intersection(b)) for a, b in zip(input_strings.USER_INPUT_LIST, input_strings.INPUT_STREAM_LIST) ]
input_strings['IF_LENGTH'] = input_strings.INCORRECT_FIXED.apply(lambda x: len(x))
input_strings['F_LENGTH'] = input_strings.FIXES.apply(lambda x: len(x))
input_strings['CORR_LENGTH'] = input_strings.CORR.apply(lambda x: len(x))
input_strings['CORR_LENGTH'] = input_strings.CORR.apply(lambda x: len(x))

In [105]:
input_strings.tail()

TEST_SECTION_ID                                         USER_INPUT  \
126827           386163  US vessels will form two separate training units.   
126828           386166                             She has a web address.   
126829           386167  These factors dictate clear limits to the conc...   
126830           386170  Its revenues last year reached 3.2 billion dol...   
126831           386173                 They have not issued an order yet.   

                                             INPUT_STREAM  \
126827  US vessels will form two separate training units.   
126828                             She has a web address.   
126829  These factors dictate clear limits to the conc...   
126830  TIta sts revenues last year reached 3.2 billio...   
126831               They hsave not issued oan order yet.   

                                        INPUT_STREAM_FULL  \
126827  ^^U^S vessels will form two separate training ...   
126828                            ^She has a web address.   
126829  ^These factors dictate clear limits to the con...   
126830  ^T<^Ita <<<s<ts revenues last year reached 3.2...   
126831            ^They hs<ave not issued o<an order yet.   

                                        INPUT_STREAM_LIST  \
126827  {a, e, w, g, f, s, p, v, S, r, u, ., U, o, t, ...   
126828                  {a, e, w, h, s, b, S, r, ., d,  }   
126829  {a, e, h, g, f, s, p, r, ., i, d, n, o, T, m, ...   
126830  {h, ., r, y, T, l, i, c,  , b, 3, 2, o, a, u, ...   
126831   {a, e, h, s, u, v, r, i, y, ., n, o, T, t, d,  }   

                                          USER_INPUT_LIST INCORRECT_FIXED  \
126827  {a, e, w, g, f, s, p, v, S, r, u, ., U, o, t, ...              {}   
126828                  {a, e, w, h, s, b, S, r, ., d,  }              {}   
126829  {a, e, h, f, s, p, r, ., i, d, n, o, T, m, l, ...             {g}   
126830  {h, ., r, y, l, i, c,  , b, 3, 2, o, a, u, v, ...             {T}   
126831   {a, e, h, s, u, v, r, i, y, ., n, o, T, t, d,  }              {}   

                                   INPUT_STREAM_FULL_LIST   FIXES  IF_LENGTH  \
126827  {w, ., r, U, l, i,  , o, m, ^, a, f, u, v, n, ...     {^}          0   
126828               {a, e, w, h, s, b, S, r, ., d, ^,  }     {^}          0   
126829  {h, ., r, T, l, i, c,  , o, <, m, ^, a, f, n, ...  {<, ^}          1   
126830  {h, ., r, y, T, l, i, c,  , b, 3, 2, <, o, ^, ...  {<, ^}          1   
126831  {a, e, h, s, u, v, r, i, y, ., d, n, <, o, T, ...  {<, ^}          0   

        F_LENGTH                                               CORR  \
126827         1  { , a, w, e, g, f, s, u, v, S, r, ., i, U, o, ...   
126828         1                  {a, w, e, h, s, b, S, r, ., d,  }   
126829         2  {a, e, h, f, ., p, r, s, d, n, o, t, T, m, l, ...   
126830         2  {h, ., r, y, l, i, c,  , b, 3, 2, o, a, u, v, ...   
126831         2   {a, e, h, s, u, v, r, ., y, n, o, t, T, i, d,  }   

        CORR_LENGTH  
126827           20  
126828           11  
126829           18  
126830           21  
126831           16

In [106]:
input_strings[['TEST_SECTION_ID','IF_LENGTH', 'F_LENGTH', 'CORR_LENGTH']].to_sql('input_strings', con=engine, if_exists='append')

In [ ]:
# Table input_strings now contains computed stuff fot the FIRST_15_TEST_SECTIONS table.
# Join it to FIRST_15_TEST_SECTIONS by TEST_SECTION_ID

"""
#ALTER TABLE FIRST_15_TEST_SECTIONS ADD COLUMN IF_LENGTH INT;

UPDATE FIRST_15_TEST_SECTIONS ts
set IF_LENGTH = (
    select 
       IF_LENGTH
    from input_strings ins
    where ins.TEST_SECTION_ID = ts.TEST_SECTION_ID
    limit 1
)


"""

# Other snippets

In [ ]:
"""
UPDATE FIRST_15_TEST_SECTIONS ts
set INPUT_STREAM_FULL = (
    select 
        REPLACE(REPLACE(INPUT_STREAM_FULL, 'SHIFT', '^'), 'BKSP', '<') AS INPUT_STREAM_FULL
    from FIRST_15_TEST_SECTIONS
)
"""

In [ ]:
## Other snippets 

"""
SELECT TEST_SECTION_ID, GROUP_CONCAT(LETTER ORDER BY PRESS_TIME SEPARATOR '')
FROM KEYSTROKES
GROUP BY TEST_SECTION_ID
"""

In [ ]:
## These speciak keys also need replacement 

"""
SELECT KEYCODE
FROM KEYSTROKES

WHERE 
   KEYCODE in ( 9,  17, 18, 19, 20, 27, 33, 34, 35, 36, 37, 38, 39, 40, 42, 44, 45, 46, 91, 93, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 144, 145)
   and 
   TEST_SECTION_ID in (
   	SELECT TEST_SECTION_ID
   	FROM FIRST_15_TEST_SECTIONS
   )
   
GROUP BY KEYCODE

# 9,  17, 18, 19, 20, 27, 36, 37, 38, 39, 40, 45, 46, 107, 112, 119, 120, 121, 122, 123, 124

+
ALT
ARw_DOWN
ARW_LEFT
ARW_RIGHT
ARW_UP
CAPS_LOCK
CTRL
DELETE
ESC
F1
F10
F11
F12
F8
HOME
INSERT
NUM_LK
PAUSE
TAB
"""

In [ ]:
# There are just 909 records with other symbols 

"""
SELECT COUNT(KEYSTROKE_ID)
FROM KEYSTROKES

WHERE 
   KEYCODE in (9,  17, 18, 19, 20, 27, 36, 37, 38, 39, 40, 45, 46, 107, 112, 119, 120, 121, 122, 123, 124)
   and 
   TEST_SECTION_ID in (
   	SELECT TEST_SECTION_ID
   	FROM FIRST_15_TEST_SECTIONS
   )
"""

In [ ]:
# Select test sections which have "unexpected" (see above) control characters 

"""
SELECT * from FIRST_15_TEST_SECTIONS
where TEST_SECTION_ID in (
SELECT DISTINCT(TEST_SECTION_ID)
FROM KEYSTROKES

WHERE 
   KEYCODE in (9, 17, 18, 19, 20, 27, 36, 37, 38, 39, 40, 45, 46, 107, 112, 119, 120, 121, 122, 123, 124)
   and 
   TEST_SECTION_ID in (
   	SELECT TEST_SECTION_ID
   	FROM FIRST_15_TEST_SECTIONS
   )
)
"""